In [ ]:
import sys
import asyncio
import aiohttp
sys.path.append('/Users/andreas/fun/useful_py_func/pyTibber/tibber/')
from tibber import Tibber
import datetime

In [ ]:
async def main():
    TOKEN = "U4L8yS_OHsfgKndAhNQZ8K-JYElbNUagYvToCF3ZPVE"
    try:
        # Initialize Tibber connection
        tibber_connection = Tibber(access_token=TOKEN)
        await tibber_connection.update_info()

        # Retrieve the websocketSubscriptionUrl
        if tibber_connection.realtime.sub_endpoint:
            print("WebSocket Subscription URL:", tibber_connection.realtime.sub_endpoint)

        # Get hourly PriceRating and PriceLevel for each active home
        for home_id in tibber_connection.get_home_ids(only_active=True):
            home = tibber_connection.get_home(home_id)
            if home is not None:
                await home.update_price_info()
                if home.current_price_info:
                    price_info = home.current_price_info
                    print(f"Home ID: {home_id}")
                    print(f"Hourly PriceRating: {price_info['total']}")
                    print(f"PriceLevel: {price_info['priceLevel']}")
                    
        # Closing the connection
        await tibber_connection.close_connection()
    except Exception as e:
        print(f"An error occurred: {e}")

    if __name__ == "__main__":
        loop = asyncio.get_event_loop()
        loop.run_until_complete(main())
        


In [ ]:
#Was outputting the basic info, but recieving error 429
async def main():
    TOKEN = "U4L8yS_OHsfgKndAhNQZ8K-JYElbNUagYvToCF3ZPVE"
    USER_AGENT = "LED_client"
    tibber_connection = Tibber(access_token=TOKEN,user_agent=USER_AGENT)
    await tibber_connection.update_info()


    # GraphQL query to fetch home information
    query = '''
    {
      viewer {
        homes {
          id
          address {
            address1
            city
          }
          owner {
            name
          }
          currentSubscription {
            priceInfo {
              current {
                total
                level
              }
            }
          }
        }
        websocketSubscriptionUrl
      }
    }
    '''

    data = await tibber_connection.execute(query)
    if data:
        homes = data['viewer']['homes']
        for home in homes:
            print(f"Home ID: {home['id']}")
            print(f"Address: {home['address']['address1']}, {home['address']['city']}")
            print(f"Owner's Name: {home['owner']['name']}")

            # Check if currentSubscription and priceInfo are not None
            if home.get('currentSubscription') and home['currentSubscription'].get('priceInfo'):
                current_price_info = home['currentSubscription']['priceInfo'].get('current')
                if current_price_info:
                    print(f"Current Power Price: {current_price_info['total']}")
                    print(f"Price Level: {current_price_info['level']}")

    await tibber_connection.close_connection()
await main()

In [ ]:
ACCESS_TOKEN = "U4L8yS_OHsfgKndAhNQZ8K-JYElbNUagYvToCF3ZPVE"
USER_AGENT = "LED_client_1"

def _callback(pkg):
    data = pkg.get("data")
    print("Real-time data received:", pkg)
    print(data)

async def run():
    async with aiohttp.ClientSession() as session:
        tibber_connection = Tibber(ACCESS_TOKEN, websession=session, user_agent=USER_AGENT)
        await tibber_connection.update_info()

        # Fetching the WebSocket Subscription URL dynamically
        query = '''
        {
          viewer {
            homes {
              id
              address {
                address1
                city
              }
              owner {
                name
              }
              currentSubscription {
                priceInfo {
                  current {
                    total
                    level
                  }
                }
              }
            }
            websocketSubscriptionUrl
          }
        }

        '''
        data = await tibber_connection.execute(query)
        if data:
            websocket_url = data['viewer']['websocketSubscriptionUrl']
            print("WebSocket Subscription URL:", websocket_url)
            print(data)
            print("data:", data['viewer']['homes'][1]['currentSubscription']['priceInfo']['current']['level'])

        # Subscribing to real-time updates for the first home
        home = tibber_connection.get_homes()[0]
        home.update_info() # I want the _callback function to execute the home.update_info() function 
        await home.rt_subscribe(_callback)    

        while True:
            await asyncio.sleep(20)
await run()


In [ ]:
ACCESS_TOKEN = "U4L8yS_OHsfgKndAhNQZ8K-JYElbNUagYvToCF3ZPVE"
USER_AGENT = "LED_client"
PRICE_QUERY_INTERVAL = 10  # Interval in seconds (e.g., 3600 seconds for hourly updates)

def _callback(pkg):
    print("Real-time data received:", pkg)

async def fetch_price_level(tibber_connection):
    i = 0
    query = '''
    {
      viewer {
        homes {
          currentSubscription {
            priceInfo {
              current {
                level
              }
            }
          }
        }
      }
    }
    '''
    while True:
        i +=1
        if i>= 3:
            break
        else:
            print('i:',i)
        
        data = await tibber_connection.execute(query)
        if data:
            for home in data['viewer']['homes']:
                if home.get('currentSubscription') and home['currentSubscription'].get('priceInfo'):
                    current_price_info = home['currentSubscription']['priceInfo'].get('current')
                    if current_price_info:
                        print(f"Current Price Level: {current_price_info['level']}")
        await asyncio.sleep(PRICE_QUERY_INTERVAL)

async def run():
    async with aiohttp.ClientSession() as session:
        tibber_connection = Tibber(ACCESS_TOKEN, websession=session, user_agent=USER_AGENT)
        await tibber_connection.update_info()

        # Start the periodic task for fetching price level
        asyncio.create_task(fetch_price_level(tibber_connection))
        
        while True:
            await asyncio.sleep(10)

await run()


In [ ]:
#import asyncio
#import datetime as dt
#import aiohttp
from tibber import Tibber, InvalidLoginError, FatalHttpExceptionError, RetryableHttpExceptionError

class MicroBitCommunicator:
    # This class can be expanded to include specific methods for micro-bit communication.
    @staticmethod
    async def update_micro_bit(price_level):
        # Implement the actual communication mechanism with micro-bit here.
        print(f"Updating micro-bit with new price level: {price_level}")
        # Add code to send data to micro-bit.

class TibberAPI:
    MAX_RETRIES = 5

    def __init__(self, access_token: str, user_agent: str = "Tibber_Client_5"):
        self._access_token = access_token
        self._user_agent = user_agent
        self._tibber_connection = None
        self.last_price_level = None
        self.retry_count = 0

    async def connect(self):
        self._session = aiohttp.ClientSession()
        self._tibber_connection = Tibber(self._access_token, websession=self._session, user_agent=self._user_agent)
        while self.retry_count < self.MAX_RETRIES:
            try:
                await self._tibber_connection.update_info()
                break
            except Exception as e:
                    self.retry_count += 1
                    print(f"Attempt {self.retry_count}/{self.MAX_RETRIES} failed: {e}")
                    await asyncio.sleep(2 ** self.retry_count)  # Exponential back-off

            if self.retry_count >= self.MAX_RETRIES:
                print("Maximum retry attempts reached. Exiting.")

    async def close_connection(self):
        if self._session:
            await self._session.close()

    async def fetch_price_info(self, home_id: str):
        i=0
        home = self._tibber_connection.get_home(home_id)
        if home is None:
            raise ValueError(f"No Tibber home found with ID: {home_id}")

        while True:
            i += 1
            if i>= 5:
                break
            else:
                print('i',i)
            try:
                print('loop, value of level:', self.last_price_level)
                if self.last_price_level is None:
                    await home.update_info()
                    print('None, new value:', home.current_price_info.get('level'))
                else:
                    next_hour = dt.datetime.now(dt.timezone.utc).replace(minute=0, second=0, microsecond=0) + dt.timedelta(hours=1)
                    delay = (next_hour - dt.datetime.now(dt.timezone.utc)).total_seconds()
                    print('Delay for next update:', round(delay), '[s]')
                    await asyncio.sleep(delay)
                    await home.update_info()

                if home.current_price_info:
                    current_price_level = home.current_price_info.get('level')
                    print(f"Current Price Level for {home_id}: {current_price_level}")
                    if current_price_level != self.last_price_level:
                        self.last_price_level = current_price_level
                        await MicroBitCommunicator.update_micro_bit(current_price_level)
            except Exception as e:
                print(f"Error in fetch_price_info: {e}")
                await asyncio.sleep(60)  # Wait a bit before retrying



async def main():
    access_token = "U4L8yS_OHsfgKndAhNQZ8K-JYElbNUagYvToCF3ZPVE"  
    home_id = "975996b6-e7ca-4fbf-9f72-61e2df95bc0c"
    tibber_api = TibberAPI(access_token)
    await tibber_api.connect()
    if tibber_api._tibber_connection:
        print(f"Connected to Tibber as {tibber_api._tibber_connection.name}")
        asyncio.create_task(tibber_api.fetch_price_info(home_id))
        try:
            while True:
                await asyncio.sleep(60)
        except KeyboardInterrupt:
            print("Stopping the program...")
        await tibber_api.close_connection()

await main()
